# Treasure Island

## Import packages

In [2]:
import numpy as np
from numpy import typing as npt

from typing import List, Tuple

In [3]:
a = np.array([[1, 2, 3],
              [4, 5, 6]])

### Set a Random Number Generator

In [4]:
rng = np.random.RandomState(42)

In [5]:
class UserInterface:
    pass

In [6]:
class MapGenerator:
    pass

In [9]:
a = np.array([[0, 1, 2],
              [3, 4, 5]])
print(a.shape)
b = np.unravel_index((0, 5), a.shape)
print(b)
c = list(zip(*b))
c

(2, 3)
(array([0, 1]), array([0, 2]))


[(0, 0), (1, 2)]

In [10]:
class Map:
    def __init__(
        self, 
        W: int, 
        H: int,
        value: np.ndarray,                          # matrix of titles's value (string)
        region: np.ndarray,                         # matrix of titles's region (int)
        scanned: np.ndarray,                        # matrix of titles that is scanned (bool)
        potential: np.ndarray,                      # matrix of titles potential (bool)
        agent: int,                                 # agent's coordinate (flattened index)
        pirate: int,                                # pirate's coordinate (flattened index)
        treasure: int,                              # treasure's coordinate (flattened index)
        total_region: int,                          # maximum number of regions
        regions: List[List[int]],                   # titles that categorize based on regions (1, 2, ..., max region)
        beaches: List[List[int]],                   # titles that are at the seaside
        boundaries: List[List[int]]                 # titles that are in boundary between two regions
    ) -> None:
        self.total_title = W * H
        self.shape = (W, H)
        self.value = value 
        self.region = region
        self.scanned = scanned
        self.potential = potential
        self.agent = agent
        self.pirate = pirate
        self.treasure = treasure
        self.total_region = total_region
        self.regions = regions
        self.beaches = beaches
        self.boundaries = boundaries

        # Map generate hints function to string
        self.hints = {"1": self.generate_hint_1, "2": self.generate_hint_2, "3": self.generate_hint_3, "4": self.generate_hint_4,
                      "5": self.generate_hint_5, "6": self.generate_hint_6, "7": self.generate_hint_7, "8": self.generate_hint_8,
                      "9": self.generate_hint_9, "10": self.generate_hint_10, "11": self.generate_hint_11, "12": self.generate_hint_12,
                      "13": self.generate_hint_13, "14": self.generate_hint_14, "15": self.generate_hint_15, "16": self.generate_hint_16}

    def hint_generator(self):
        np.random.randint(16)

    
    def generate_hint(self) -> None:
        self.hints[str(rng.randint(16))]()
    
    def generate_hint_1(self) -> Tuple[bool, List[Tuple[int, int]], str]:
        # A list of random tiles that doesn't contain the treasure (1 to 12)

        # trueness of this hint
        trueness = True

        # get random tiles doest not contain the treasure
        no_titles = rng.randint(1, 13)
        rand_titles = rng.choice(np.arange(self.total_title), size=no_titles, replace=False)

        # get the list of tiles that overlap with the treasure
        overlaps = rand_titles == self.treasure
        if overlaps.any():
            trueness = False

        # mark those cells that they are not contain the treasure
        title_coords = np.unravel_index(rand_titles, self.shape)
        if trueness:
            self.scanned[title_coords] = True
        else:
            self.potential[title_coords] = True
            masked_titles = np.zeros(self.shape, dtype=bool)
            masked_titles[title_coords] = True
            self.scanned[masked_titles] = True

        hinted_titles = list(zip(*title_coords))

        log = f"These tiles {hinted_titles} do not contain the treasure"
                        
        return trueness, hinted_titles, log
        
    def generate_hint_2(self) -> Tuple[bool, List[int], str]:
        # 2-5 regions that 1 of them has the treasure.

        # trueness of this hint
        trueness = rng.randint(2, dtype=bool)

        # number of regions
        no_reg = rng.randint(1, 5)
        regions = rng.choice(np.arange(1, self.total_region + 1), size=no_reg, replace=False)
 
        # if the hint is true then at least one of these regions contains the treasure
        if trueness:
            treasure_coord = np.unravel_index(self.treasure, self.shape)
            regions[0] = self.region[treasure_coord]

        for i in regions:
            for cell in self.regions[i]:
                if trueness:
                    cell.is_potential = True
                else:
                    cell.is_scanned = True

        if trueness:
            



        regions = list(regions)
        
        log = f"One of these regions contain the treasure: {regions}"
            
        return trueness, regions, log

    def generate_hint_3(self) -> Tuple[bool, List[int], str]:
        # 1-3 regions that do not contain the treasure.

        # trueness of this hint
        trueness = rng.randint(2, dtype=bool)

        # number of regions
        no_reg = rng.randint(1, 3)
        regions = rng.choice(np.arange(1, self.max_region + 1), size=no_reg, replace=False)

        # number of regions
        if not trueness:
            regions[0] = self.treasure.region

        for i in regions:
            for cell in self.regions[i]:
                if not trueness:
                    cell.is_potential = True
                else:
                    cell.is_scanned = True
        
        log = f"These regions do not contain the treasure: {regions}"

        return trueness, list(regions), log

    def generate_hint_4(self) -> List[Cell]:
        pass

    def generate_hint_5(self) -> List[Cell]:
        pass

    def generate_hint_6(self) -> Tuple[bool, None, str]:
        # You are the nearest person to the treasure

        agent_treasure = abs(sum(self.agent.coord) ** 2 - sum(self.treasure.coord) ** 2)
        pirate_treasure = abs(sum(self.pirate.coord) ** 2 - sum(self.treasure.coord) ** 2)

        # trueness of this hint
        trueness = agent_treasure > pirate_treasure

        log = "You are the nearest person to the treasure"

        return trueness, None, log

    def generate_hint_7(self) -> List[Cell]:
        pass

    def generate_hint_8(self) -> List[Cell]:
        pass

    def generate_hint_9(self) -> List[Cell]:
        pass

    def generate_hint_10(self) -> List[Cell]:
        pass

    def generate_hint_11(self) -> List[Cell]:
        pass

    def generate_hint_12(self) -> List[Cell]:
        pass

    def generate_hint_13(self) -> List[Cell]:
        pass

    def generate_hint_14(self) -> List[Cell]:
        # From the center of the map/from the prison that he's staying, he tells you a direction that has the treasure

        # center of the map

        center_point = 

        pass

    def generate_hint_15(self) -> List[Cell]:
        pass

    def generate_hint_16(self) -> List[Cell]:
        pass

NameError: name 'Cell' is not defined

In [ ]:
class Agent:
    def __init_(self) -> None:
        self.position = Cell()
        
        pass

In [ ]:
class JackSparrow(Agent):
    def __init__(self) -> None:
        pass
    
    def action(self):
        pass

    def move(self, step: int) -> None:
        pass

    def verifies_hint(self) -> bool:
        pass

    def small_scan(self) -> None:
        pass

    def large_scan(self) -> None:
        pass

    def teleport(self) -> None:
        pass

    def is_treasure(self) -> bool:
        pass

In [ ]:
class Hint:
    def __init__(self, trueness: bool) -> None:
        self.trueness = trueness

        pass



In [ ]:
class Pirate(Agent):
    def __init__(self) -> None:
        super().__init__()
        path = self.shorted_path()
        hints = []
        pass

    def shorted_path(self) -> List[Tuple[int, str]]:
        pass

    def gives_hint(self) -> None:
        pass